In [1]:
import dolfin as dl
import numpy as np
from time import time

# https://github.com/NickAlger/helper_functions/
from csr_fenics2scipy import csr_fenics2scipy
from csr_scipy2fenics import csr_scipy2fenics

# Make Laplacian matrix in fenics

In [2]:
n = 10
mesh = dl.UnitCubeMesh(n,n,n)
V = dl.FunctionSpace(mesh, 'CG', 1)

u = dl.TrialFunction(V)
v = dl.TestFunction(V)
a = dl.inner(dl.grad(u), dl.grad(v)) * dl.dx + u * v * dl.dx

f = dl.Function(V)
f.vector()[:] = np.random.randn(V.dim())
b = f * v * dl.dx

b_fenics = dl.assemble(b)
b_numpy = b_fenics[:]

A_fenics = dl.assemble(a)

# Test correctness of matrix converters

In [3]:
A_scipy = csr_fenics2scipy(A_fenics)
A_fenics2 = csr_scipy2fenics(A_scipy)

z_fct = dl.Function(V)
z_fct.vector().set_local(np.random.randn(V.dim()))
z = z_fct.vector()
Az = A_fenics * z
Az2 = A_fenics2 * z
err = np.linalg.norm((Az - Az2)[:])/np.linalg.norm(Az[:])
print('err=', err)

err= 0.0


# Compare solve timing: fenics vs scipy

In [4]:
import scipy.sparse.linalg as spla

t = time()
solve_A_scipy = spla.factorized(A_scipy)
scipy_factorization_time = time() - t
print('scipy_factorization_time=', scipy_factorization_time)

t = time()
x_numpy = solve_A_scipy(b_numpy)
scipy_solve_time = time() - t
print('scipy_solve_time=', scipy_solve_time)

t = time()
solve_A_fenics = dl.LUSolver(A_fenics)
fenics_factorization_time = time() - t
print('fenics_factorization_time=', fenics_factorization_time)

x = dl.Function(V)

t = time()
solve_A_fenics.solve(x.vector(), b_fenics)
fenics_first_solve_time = time() - t
print('fenics_first_solve_time=', fenics_first_solve_time)

x2 = dl.Function(V)

t = time()
solve_A_fenics.solve(x2.vector(), b_fenics)
fenics_second_solve_time = time() - t
print('fenics_second_solve_time=', fenics_second_solve_time)


scipy_factorization_time= 0.014922857284545898
scipy_solve_time= 0.0005962848663330078
fenics_factorization_time= 0.00019240379333496094
fenics_first_solve_time= 0.02410602569580078
fenics_second_solve_time= 0.0006530284881591797


/home/nick/anaconda3/envs/fenics3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
